In [22]:
from haystack_integrations.document_stores.elasticsearch import ElasticsearchDocumentStore
from haystack import Pipeline
from haystack.components.embedders import OpenAITextEmbedder
from haystack_integrations.components.retrievers.elasticsearch import ElasticsearchEmbeddingRetriever
from haystack.utils import Secret
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator


from dotenv import load_dotenv
import os



load_dotenv(".env")
open_ai_key = os.environ.get("OPENAI_API_KEY")

document_store = ElasticsearchDocumentStore(hosts = "http://localhost:9200")


retriever = ElasticsearchEmbeddingRetriever(document_store=document_store)
text_embedder = OpenAITextEmbedder(api_key=Secret.from_token(open_ai_key))



template = """
Given the following information, analyze the information and generate a brief summary that addresses the question, 
at the end of your summary, ensure that you list the URLs of the documents that are relevant to the question.

Ensure that you also list the list of stock symbols that are mentioned in the documents.

Context:
Context:
{% for doc in documents %}
    Document: {{ doc.content }} 
    Headline: {{ doc.meta['headline'] }}  
    Summary {{doc.meta['summary']}} 
    Stock symbols {{doc.meta['symbols']}}
    URLs {{doc.meta['url']}}
    \n
{% endfor %};

Question: {{question}}
Answer:
"""

prompt_builder = PromptBuilder(template=template)
generator = OpenAIGenerator(api_key=Secret.from_token(open_ai_key), model="gpt-3.5-turbo")




In [23]:
query_pipeline = Pipeline()
query_pipeline.add_component("text_embedder", text_embedder)
query_pipeline.add_component("retriever", retriever)
query_pipeline.add_component("prompt_builder", prompt_builder)
query_pipeline.add_component("llm", generator)

query_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
query_pipeline.connect("retriever", "prompt_builder.documents")
query_pipeline.connect("prompt_builder", "llm")




🚅 Components
  - text_embedder: OpenAITextEmbedder
  - retriever: ElasticsearchEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [27]:
from IPython.display import display, Markdown, Latex

result = query_pipeline.run({"text_embedder": {"text": "How was stock impacted by the actions of Elon Musk"}})


display(Markdown(result['llm']['replies'][0]))
